In [1]:
cd ..

D:\Kevin\Machine Learning\Cassava Leaf Disease Classification


In [7]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm

from src.config import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
train = pd.read_csv(TRAIN)
train

,image_id,label
0,1000015157.jpg,0
1,1000201771.jpg,3
2,100042118.jpg,1
3,1000723321.jpg,1
4,1000812911.jpg,3
...,...,...
21392,999068805.jpg,3
21393,999329392.jpg,3
21394,999474432.jpg,1
21395,999616605.jpg,4


In [10]:
train.label.value_counts()

3    13158
4     2577
2     2386
1     2189
0     1087
Name: label, dtype: int64